In [1]:
import matplotlib
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

#
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#
import numpy as np
import os

#
import glob


Autosaving every 180 seconds


<ipython-input-1-94c414df1c90>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [23]:
########################################################
######### LOAD RAW DATA - ANATOMICAL AREA BASED ########
########################################################
#
root_dir = 'data'
mouse_id = 'IA1'
session_id = 'Feb_16'

# load the raw data from each neural area
animal_dir = os.path.join(root_dir, mouse_id,
                          session_id)

# find the file using glob that has "wholestack.npz" in it
fname = glob.glob(os.path.join(animal_dir, '*wholestack.npz'))[0]

#
data = np.load(fname, allow_pickle=True)
ca_data = data['whole_stack'] 
print ("ca_data [n_timesteps, n_neural_areas]: ", ca_data.shape)

#
areas = data['areas']
print (areas.shape)

#
areanames_area = data['areanames_area']
print (areanames_area.shape)

#
rois_masks = data['A_reshape'].transpose(2,0,1)
print ("Rois shapes: ", rois_masks.shape)

#
names = data['names']
print (names.shape)


# load reward_times from .txt file:
reward_times = np.loadtxt(os.path.join(root_dir,
                                    mouse_id,
                                        session_id,
                                'rewarded_times.txt'))
print (" reward_times: ", reward_times.shape)

# load the temporal shift between reward times and ca data
d = np.load(os.path.join(root_dir,
                        mouse_id,
                            session_id,
                        'blue_light_frame_triggers.npz'),
                   allow_pickle=True)

shift = d['start_blue']/30
                             
print ("shift between neural activity and behavior(seconds) ", shift)

# shift reward times by shift
reward_times = reward_times + shift



ca_data [n_timesteps, n_neural_areas]:  (16, 40000)
(16,)
(16,)
Rois shapes:  (16, 128, 128)
(16,)
 reward_times:  (43,)
shift between neural activity and behavior(seconds)  2.5


In [24]:
############################################### 
################## PLOT ROI MASKS #############
############################################### 
plt.figure(figsize=(10,10)) 

for k in range(16):
    plt.subplot(4,4,k+1)
    plt.imshow(rois_masks[k], 
               aspect='auto',
               cmap='jet')
    plt.xticks([])
    plt.yticks([])

    text = names[k]

    # break up text into 2 lines half way through
    text = text[:int(len(text)/2)] + '\n' + text[int(len(text)/2):]

    plt.title(text, fontsize=10,
               pad=0.1)

#
plt.show()

In [25]:
###################################################### 
################## PLOT ROI TIME COURSES #############
###################################################### 
plt.figure(figsize=(10,10)) 

#
t = np.arange(ca_data.shape[1])/30

#
for k in range(16):
    #
    temp = ca_data[k]

    # remove mean
    temp = temp - np.mean(temp)

    # zero out first and last 500 frames
    temp[:500] = 0
    temp[-1000:] = 0

    plt.plot(t,temp + k*500, linewidth=1)

#    
plt.xlim(t[0], t[-1])

# make the y labels for each plt.plot line using the names
plt.yticks(np.arange(0, 16*500, 500), names, fontsize=8, rotation=60)

#
plt.xlabel('Time (seconds)', fontsize=14)

# 
plt.suptitle("Mouse: "+ mouse_id + " , session: " + session_id, fontsize=14)

#
plt.show()

In [26]:
###################################################
##### VISUALIZE FULL SESSION + REWARD TIMES #######
###################################################

#
window_size = 10 *30   # 10 seconds

ctr=0
t = np.arange(ca_data.shape[0])/30.
stacks = []
for k in range(16):
    temp = ca_data[k]
    temp = temp-temp.mean()
    temp[:300]=0
    temp[-1000:]=0

    # grab a stack of 300 frames around the reward time from temp
    stack = []
    for p in range(reward_times.shape[0]):
        stack.append(temp[int(reward_times[p]*30)-window_size:int(reward_times[p]*30)+window_size])

    # make into an array
    stack = np.array(stack)
   #print (k, stack.shape)
    stacks.append(stack)
    
###############################################
# plot average stacks for first value
plt.figure(figsize=(12,12))

#
for area_id in range(16):
    #
    plt.subplot(4,4,area_id+1)

    #
    text = names[area_id]

    # break up text into 2 lines half way through
    text = text[:int(len(text)/2)] + '\n' + text[int(len(text)/2):]

    plt.title("Area "+text,fontsize=10)

    #
    plt.yticks([])

    if area_id>=12:
        plt.xlabel("Time (sec)",fontsize=12)
    else:
        plt.xticks([])

    #
    temp = stacks[area_id]
    #print ("temp: ", temp.shape)
    t = np.arange(temp.shape[1])/30-window_size/30

    # grab mean and std
    mean = temp.mean(0)
    std = temp.std(0)

    #
    plt.plot(t,mean, c='black')

    # plot std as shaded area
    plt.fill_between(t, mean-std, mean+std, 
                     color='black',
                     alpha=.2)

    # plot horizontal line at 0
    plt.plot([t[0], t[-1]], [0,0],'r--')


    # get max and ymin
    ymin, ymax = plt.ylim()

    # plot vertical line at 0
    plt.plot([0,0],[ymin,ymax],'r--')

plt.show()

In [44]:
bb = np.load('/home/cat/code/neural_transformer/data/IA1/May_4/blue_light_frame_triggers.npz', allow_pickle=True)
start_blue = bb['start_blue']

print ("start_blue: ", start_blue)

fname = '/home/cat/code/neural_transformer/data/IA1/May_4/start_blue.npy'
sb = np.load(fname, allow_pickle=True)

print ("sb: ", sb)

start_blue:  56
sb:  56
